# Databricks SQL
<br>

<div style="float: right; width: 100%;">
  <img 
    src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/dbsql/sql-etl-hls-patient/Databricks%20SQL%20Intro.png?raw=true" 
    width="100%"
  >
</div>

<br>

# End-to-End Data Warehousing Solution
<br>

<div style="float: right; width: 100%;">

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/dbsql/sql-etl-hls-patient/Databricks%20SQL%20Marketecture.png?raw=true" style="float: right" width="100%">

</div>

# SQL Centric Engine Capabilities

Let's first start with some examples of Databricks SQL capabilities before moving onto the full-fledged example of Creating and Populating a dimension in a Star Schema.

## 1. Declare Variable

Variables are typed objects which store values that are private to a session. In Databricks variables are temporary and declared within a session using the DECLARE VARIABLE statement.

Public Documentation: [AWS](https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-syntax-ddl-declare-variable) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/sql-ref-syntax-ddl-declare-variable) | [GCP](https://docs.databricks.com/gcp/en/sql/language-manual/sql-ref-syntax-ddl-declare-variable)

In [0]:
-- Storage location for demo sample files
DECLARE OR REPLACE VARIABLE cloud_loc STRING;
SET VARIABLE cloud_loc = "s3://dbdemos-dataset/dbsql/sql-etl-hls-patient";

-- List of sample patient data file names
DECLARE OR REPLACE VARIABLE file_names ARRAY<STRING>;
SET VARIABLE file_names = array('patients50_init.csv', 'patients_incr1.csv', 'patients_incr2.csv', 'NOTICE.txt');

## 2. SQL Scripting

- Write complex logic with Pure SQL
- Combine SQL Statements with Control Flow 
- SQL/PSM standard
- Refer to Public Documentation: [AWS](https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-scripting) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/sql-ref-scripting) | [GCP](https://docs.databricks.com/gcp/pt/sql/language-manual/sql-ref-scripting)

![](https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/dbsql/sql-etl-hls-patient/SQL%20Scripting.png?raw)

### Example

The SQL Script below does the following:<br>

Loop through the 3 patient sample data files to-
1. Identify the headers
2. Create persistent views in the current schema

<u>NOTE</u>:
The SQL script below will create or **replace** any existing views with the following names, in the current schema:
- dbdemos_patients50_init
- dbdemos_patients_incr1
- dbdemos_patients_incr2

In [0]:
-- Explore format of sample patient data files
BEGIN
  DECLARE first_row STRING;
  DECLARE result_arr ARRAY<STRING> DEFAULT array();
  DECLARE sqlstr STRING;
  DECLARE view_name STRING;

  -- Drop the 3 views if existing
  -- These are created in this script
  DROP VIEW IF EXISTS dbdemos_patients50_init;
  DROP VIEW IF EXISTS dbdemos_patients_incr1;
  DROP VIEW IF EXISTS dbdemos_patients_incr2;

  lbl_for_loop:
  FOR filelist AS (SELECT explode(file_names) AS file_name)
  DO
    -- For each file (name), do

    -- Read first line from file into variable
    SET first_row = (SELECT * FROM read_files(cloud_loc || '/' ||  filelist.file_name, format=>'TEXT') LIMIT 1);

    IF (first_row IS NOT NULL)
    THEN

      IF startswith(first_row, 'Id,')
      THEN
        -- First line of file starts with Id column
        -- Record this in result_arr
        SET result_arr = array_append(result_arr, filelist.file_name || ' header: \n' || first_row);

        -- Create view for ease of data exploration
        SET view_name = 'dbdemos_' || rtrim('.csv', filelist.file_name);

        -- Note: Any temp view is limited to current scope
        -- Below EXECUTE will create persistent view in the CURRENT SCHEMA
        SET sqlstr = 'CREATE VIEW ' || view_name ||
          ' AS SELECT * FROM read_files(\'' || cloud_loc || '/' ||  filelist.file_name || '\')';
        EXECUTE IMMEDIATE sqlstr;
      ELSE
        SET result_arr = array_append(result_arr, filelist.file_name || ': ' || 'Header not present');
      END IF;
      
    ELSE
      -- file empty
      SET result_arr = array_append(result_arr, filelist.file_name || ': ' || 'File empty');
    END IF;

  END FOR lbl_for_loop;

  -- Show results for csv file header
  SELECT explode(result_arr) AS result;
 END;

In [0]:
-- Browse data from the 3 views created in the script
SELECT * FROM dbdemos_patients50_init;
SELECT * FROM dbdemos_patients_incr1;
SELECT * FROM dbdemos_patients_incr2;

In [0]:
-- Drop the 3 views created in the script
DROP VIEW IF EXISTS dbdemos_patients50_init;
DROP VIEW IF EXISTS dbdemos_patients_incr1;
DROP VIEW IF EXISTS dbdemos_patients_incr2;

## Ok you're now ready, let's get started with a full example 

This example will create and populate a SCD Type 2 dimension using Databricks SQL.

[Patient Dimension ETL Introduction]($./01-patient-dimension-ETL-introduction)